In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [24]:
import pandas as pd
import numpy as np

# --- 1. Configuration: Columns to Keep ---
# We will "delete" all columns *except* these by only loading these.
COLUMNS_TO_KEEP_2019 = [
    'Respondent',
    'Country',
    'Employment',
    'DevType',
    'YearsCodePro',
    'ConvertedComp',
    'LanguageDesireNextYear',
    'DatabaseDesireNextYear',
    'PlatformDesireNextYear',
    'WebFrameDesireNextYear',
    'MiscTechDesireNextYear'
]

# --- 2. Configuration: Row Filters ---
# NEW: Filter for a list of major tech countries
# We'll need to use the exact strings from the 2019 survey
TARGET_COUNTRIES = [
    'United States',
'India',
'Germany',
'United Kingdom',
'Canada',
'France',
'Brazil',
'Poland',
'Australia',
'Netherlands',
'Russian Federation',
'Spain',
'Italy',
'Sweden',
'Switzerland',
'Israel',
'Turkey',
'Pakistan',
'Ukraine',
'Austria',
'Czech Republic',
'Romania',
'Iran',
'Belgium',
'China',
'Bulgaria',
'Mexico',
'South Africa',
'Denmark',
'Bangladesh',
'Norway',
'Greece',
'Argentina',
'Finland',
'Portugal',
'New Zealand',
'Nigeria',
'Hungary',
'Indonesia',
'Ireland'
]
EMPLOYMENT_STATUS = 'Employed full-time'
YEARS_PRO_MAX = 4  # For our "graduate / early career" audience

# --- 3. Configuration: Role Categorization Keywords ---
# This is our robust, prioritized list
ROLE_CATEGORIES = {
    'Developer': 'Developer, full-stack|Developer, back-end|Developer, front-end|Developer, desktop|Developer, mobile|Developer, embedded|Developer, QA|Developer, game',
    'Data': 'Data scientist|Engineer, data|Data or business analyst',
    'DevOps': 'DevOps specialist|Engineer, site reliability|Database administrator|System administrator'
}

# --- 4. Helper Function for Cleaning ---
def clean_yearscodepro(value):
    """Converts the 'YearsCodePro' text column to a clean number."""
    if pd.isna(value) or value == 'NA':
        return None
    if value == 'Less than 1 year':
        return 0
    if value == 'More than 50 years':
        return 50
    try:
        return float(value)
    except ValueError:
        return None

# --- 5. Main Cleaning Script ---

def clean_2019_data(file_path='data/survey_results_public2019.csv'):
    """
    Loads, cleans, and filters the 2019 survey data using
    intelligent, prioritized role categorization.
    """
    print(f"Starting 2019 data cleaning process for {file_path}...")
    
    try:
        # --- Step 1: Load Data & "Delete" Columns ---
        df = pd.read_csv(
            file_path,
            usecols=COLUMNS_TO_KEEP_2019,
            low_memory=False
        )
        print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns.")

        # --- Step 2: "Delete" Irrelevant Rows (Initial Pass) ---
        # Drop rows missing key data *before* filtering
        df = df.dropna(subset=[
            'Country', 
            'Employment', 
            'DevType', 
            'YearsCodePro', 
            'ConvertedComp'
        ])
        
        # --- NEW: Apply the multi-country filter ---
        df = df[df['Country'].isin(TARGET_COUNTRIES)].copy()
        print(f"Shape after filtering for {len(TARGET_COUNTRIES)} countries: {df.shape}")

        # Apply other filters
        df = df[df['Employment'] == EMPLOYMENT_STATUS]
        print(f"Shape after filtering for Employment '{EMPLOYMENT_STATUS}': {df.shape}")
        
        # --- Step 3: Prioritized Role Categorization ---
        print("Applying prioritized role categorization...")
        
        conditions = [
            df['DevType'].str.contains(ROLE_CATEGORIES['Developer'], na=False),
            df['DevType'].str.contains(ROLE_CATEGORIES['Data'], na=False),
            df['DevType'].str.contains(ROLE_CATEGORIES['DevOps'], na=False)
        ]
        
        choices = [
            'Developer',
            'Data',
            'DevOps'
        ]
        
        df['Role_Category'] = np.select(conditions, choices, default='Other')
        
        # Now, "delete" rows that didn't match our 3 key categories
        df = df[df['Role_Category'].isin(['Developer', 'Data', 'DevOps'])]
        
        print(f"Shape after retaining 'Developer', 'Data', & 'DevOps' roles: {df.shape}")

        # --- Step 4: Clean Remaining Data ---
        print("Cleaning remaining columns (YearsCodePro, ConvertedComp)...")
        
        df['YearsCodePro_Numeric'] = df['YearsCodePro'].apply(clean_yearscodepro)
        df['ConvertedComp'] = pd.to_numeric(df['ConvertedComp'], errors='coerce')

        df = df.dropna(subset=['YearsCodePro_Numeric', 'ConvertedComp'])
        print(f"Shape after cleaning numeric columns: {df.shape}")

        # --- Step 5: Apply Final "Graduate" Filter ---
        df_cleaned = df[df['YearsCodePro_Numeric'] <= YEARS_PRO_MAX].copy()
        print(f"Shape after filtering for early-career (<= {YEARS_PRO_MAX} years): {df_cleaned.shape}")

        # --- Step 6: Final Column Cleanup ---
        # We "delete" the original messy columns and keep the new clean ones
        df_cleaned = df_cleaned.drop(columns=['DevType', 'YearsCodePro', 'YearsCodePro_Numeric'])

        # --- Finish ---
        print("\n--- Cleaning Complete ---")
        print(f"Final shape: {df_cleaned.shape}")
        
        print("\nFinal DataFrame Info:")
        df_cleaned.info()
        
        print("\nFinal DataFrame Head:")
        print(df_cleaned.head())

        # Save the fully cleaned data to a new file
        output_filename = 'major_cleaned_2019_survey.csv'
        df_cleaned.to_csv(output_filename, index=False)
        print(f"\nSuccessfully cleaned data and saved to {output_filename}")
        
        return df_cleaned

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'.")
        print("Please check the path and file name.")
    except KeyError as e:
        print(f"\nError: A required column is missing: {e}")
        print("The CSV file does not contain all the columns specified in `COLUMNS_TO_KEEP_2019`.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

# --- To run the script ---
# 1. Make sure you have the file 'data/survey_results_public2019.csv'
# 2. Then, uncomment and run the line below:
clean_2019_data('data/survey_results_public2019.csv')

Starting 2019 data cleaning process for data/survey_results_public2019.csv...
Loaded 88883 rows and 11 columns.
Shape after filtering for 40 countries: (49727, 11)
Shape after filtering for Employment 'Employed full-time': (44062, 11)
Applying prioritized role categorization...
Shape after retaining 'Developer', 'Data', & 'DevOps' roles: (43157, 12)
Cleaning remaining columns (YearsCodePro, ConvertedComp)...
Shape after cleaning numeric columns: (43157, 13)
Shape after filtering for early-career (<= 4 years): (17249, 13)

--- Cleaning Complete ---
Final shape: (17249, 10)

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 17249 entries, 3 to 88325
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Respondent              17249 non-null  int64  
 1   Employment              17249 non-null  object 
 2   Country                 17249 non-null  object 
 3   ConvertedComp           

#### 2024 file

In [31]:
import pandas as pd

def clean_2024_tech_roles(file_path='data/survey_results_public2024.csv'):
    print(f"Starting filtered 2024 tech role extraction for {file_path}...")

    # --- 1. Target countries ---
    TARGET_COUNTRIES_2024 = [
        'United States of America',
        'India',
        'Germany',
        'United Kingdom of Great Britain and Northern Ireland',
        'Canada',
        'France',
        'Brazil',
        'Poland',
        'Australia',
        'Netherlands',
        'Russian Federation',
        'Spain',
        'Italy',
        'Sweden',
        'Switzerland',
        'Israel',
        'Turkey',
        'Pakistan',
        'Ukraine',
        'Austria',
        'Czech Republic',
        'Romania',
        'Iran, Islamic Republic of...',
        'Belgium',
        'China',
        'Bulgaria',
        'Mexico',
        'South Africa',
        'Denmark',
        'Bangladesh',
        'Norway',
        'Greece',
        'Argentina',
        'Finland',
        'Portugal',
        'New Zealand',
        'Nigeria',
        'Hungary',
        'Indonesia',
        'Ireland'
    ]

    # --- 2. Tech-related columns ---
    TECH_COLS_2024 = [
        'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired',
        'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired',
        'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired',
        'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired',
        'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired',
        'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired',
        'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired',
        'AISearchDevHaveWorkedWith', 'AISearchDevWantToWorkWith', 'AISearchDevAdmired','CompTotal'
    ]

    # --- 3. Role Mapping ---
    ROLE_MAP_2024 = {
        # Developer roles
        'Developer, full-stack': 'Developer',
        'Developer, back-end': 'Developer',
        'Developer, front-end': 'Developer',
        'Developer, desktop or enterprise applications': 'Developer',
        'Developer, mobile': 'Developer',
        'Developer, embedded applications or devices': 'Developer',
        'Developer, game or graphics': 'Developer',
        'Developer, AI': 'Developer',
        'Developer, QA or test': 'Developer',
        'Developer Experience': 'Developer',
        'Developer Advocate': 'Developer',
        'Hardware Engineer': 'Developer',
        'Designer': 'Developer',

        # Data roles
        'Data engineer': 'Data',
        'Data scientist or machine learning specialist': 'Data',
        'Data or business analyst': 'Data',

        # Others
        'DevOps specialist': 'Other',
        'Cloud infrastructure engineer': 'Other',
        'System administrator': 'Other',
        'Engineer, site reliability': 'Other',
        'Database administrator': 'Other',
        'Security professional': 'Other',
        'Student': 'Other',
        'Engineering manager': 'Other',
        'Academic researcher': 'Other',
        'Research & Development role': 'Other',
        'Senior Executive (C-Suite, VP, etc.)': 'Other',
        'Scientist': 'Other',
        'Product manager': 'Other',
        'Project manager': 'Other',
        'Educator': 'Other',
        'Blockchain': 'Other',
        'Marketing or sales professional': 'Other',
        'Other (please specify):': 'Other',
        'NA': 'Other',
        '(blank)': 'Other'
    }

    try:
        # --- 4. Load full dataset ---
        df = pd.read_csv(file_path, low_memory=False)
        print(f"Loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns.")

        # --- 5. Filter by country ---
        if 'Country' not in df.columns:
            raise KeyError("Column 'Country' not found in dataset.")
        df = df[df['Country'].isin(TARGET_COUNTRIES_2024)]
        print(f"After filtering for {len(TARGET_COUNTRIES_2024)} countries: {df.shape}")

        # --- 6. Ensure DevType column exists ---
        if 'DevType' not in df.columns:
            raise KeyError("Column 'DevType' not found in dataset.")

        # Map roles into 3 categories
        df['Role_Category'] = df['DevType'].map(ROLE_MAP_2024).fillna('Other')

        # --- 7. Select required columns ---
        selected_cols = ['Country', 'DevType', 'Role_Category'] + [c for c in TECH_COLS_2024 if c in df.columns]
        df_selected = df[selected_cols].copy()

        # Fill missing values with 'NA' (for all tech columns)
        df_selected = df_selected.fillna('NA')

        print(f"Final filtered shape: {df_selected.shape}")
        print(f"Preview of cleaned data:\n{df_selected.head(3)}")

        # --- 8. Save cleaned dataset ---
        output_filename = 'major_cleaned_2024_survey.csv'
        df_selected.to_csv(output_filename, index=False)
        print(f"Saved cleaned dataset to {output_filename}")

        return df_selected

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'.")
    except Exception as e:
        print(f"Unexpected error: {e}")
clean_2024_tech_roles('data/survey_results_public2024.csv')

Starting filtered 2024 tech role extraction for data/survey_results_public2024.csv...
Loaded dataset with 65437 rows and 114 columns.
After filtering for 40 countries: (51805, 114)
Final filtered shape: (51805, 28)
Preview of cleaned data:
                                             Country                DevType  \
0                           United States of America                     NA   
1  United Kingdom of Great Britain and Northern I...  Developer, full-stack   
2  United Kingdom of Great Britain and Northern I...   Developer Experience   

  Role_Category                             LanguageHaveWorkedWith  \
0         Other                                                 NA   
1     Developer  Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...   
2     Developer                                                 C#   

                              LanguageWantToWorkWith  \
0                                                 NA   
1  Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS.

,Country,DevType,Role_Category,LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,DatabaseAdmired,PlatformHaveWorkedWith,...,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,MiscTechAdmired,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,ToolsTechAdmired,AISearchDevHaveWorkedWith,AISearchDevWantToWorkWith,AISearchDevAdmired,CompTotal
0,United States of America,NA,Other,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,United Kingdom of Great Britain and Northern I...,"Developer, full-stack",Developer,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Dynamodb;MongoDB;PostgreSQL,PostgreSQL,PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,...,NA,NA,NA,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,NA,NA,NA,NA
2,United Kingdom of Great Britain and Northern I...,Developer Experience,Developer,C#,C#,C#,Firebase Realtime Database,Firebase Realtime Database,Firebase Realtime Database,Google Cloud,...,.NET (5+) ;.NET Framework (1.0 - 4.8);.NET MAUI,.NET (5+) ;.NET Framework (1.0 - 4.8);.NET MAUI,.NET (5+) ;.NET Framework (1.0 - 4.8);.NET MAUI,MSBuild,MSBuild,MSBuild,NA,NA,NA,NA
3,Canada,"Developer, full-stack",Developer,C;C++;HTML/CSS;Java;JavaScript;PHP;PowerShell;...,HTML/CSS;Java;JavaScript;PowerShell;Python;SQL...,HTML/CSS;Java;JavaScript;PowerShell;Python;SQL...,MongoDB;MySQL;PostgreSQL;SQLite,MongoDB;MySQL;PostgreSQL,MongoDB;MySQL;PostgreSQL,Amazon Web Services (AWS);Fly.io;Heroku,...,NumPy;Pandas;Ruff;TensorFlow,NA,NA,Docker;npm;Pip,Docker;Kubernetes;npm,Docker;npm,NA,NA,NA,NA
4,Norway,"Developer, full-stack",Developer,C++;HTML/CSS;JavaScript;Lua;Python;Rust,C++;HTML/CSS;JavaScript;Lua;Python,C++;HTML/CSS;JavaScript;Lua;Python,PostgreSQL;SQLite,PostgreSQL;SQLite,PostgreSQL;SQLite,NA,...,NA,NA,NA,APT;Make;npm,APT;Make,APT;Make,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65413,India,Student,Other,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
65415,India,System administrator,Other,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
65428,India,"Developer, back-end",Developer,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
65431,Belgium,"Developer, full-stack",Developer,Bash/Shell (all shells);C;C#;C++;Delphi;Groovy...,Bash/Shell (all shells);C#;Go;HTML/CSS;Java;Ja...,Bash/Shell (all shells);C#;HTML/CSS;Java;JavaS...,Couch DB;H2;Microsoft SQL Server;MySQL;Oracle;...,Elasticsearch;PostgreSQL;Redis,PostgreSQL,Microsoft Azure,...,.NET (5+) ;.NET Framework (1.0 - 4.8);Cordova;...,.NET (5+) ;.NET Framework (1.0 - 4.8);Quarkus;...,.NET (5+) ;.NET Framework (1.0 - 4.8);Spring F...,Chocolatey;Docker;Gradle;Homebrew;Kubernetes;n...,Ansible;Chocolatey;Docker;Gradle;Homebrew;Kube...,Chocolatey;Docker;Gradle;Homebrew;Kubernetes;n...,NA,NA,NA,55000.0
